#### Libraries

In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from requests_html import HTMLSession

from tqdm.notebook import tqdm

In [2]:
#Read excel files and create the dfs
camel_df = pd.read_excel('../Data/1-s2.0-S1096717618302258-mmc1.xlsx', header = 1)
hefzi_df = pd.read_excel('../Data/iCHOv1_Reconstruction.xlsx', header = 1)
fouladiha_df = pd.read_excel('../Data/10529_2020_3021_MOESM1_ESM.xlsx', 'Supplementary Table 10', header = 1)
yeo_df = pd.read_excel('../Data/Supplementary Data.xlsx', 'Data S2')

In [3]:
#Standarization of the columns names
camel_df.rename(columns = {'Reaction ID':'Reaction', 'Initial reaction in model':'Reaction Formula', 'Reaction name':'Reaction Name', 'Justification':'Curation Notes'}, inplace = True)
fouladiha_df.rename(columns = {'Abbreviation':'Reaction', 'Description':'Reaction Name', 'Reaction':'Reaction Formula'}, inplace = True)
yeo_df.rename(columns = {'Rxn':'Reaction', 'Subsystem (iCHO1766)':'Subsystem'}, inplace = True)

# Addition of tag columns
camel_df.insert(loc=0, column='cam', value='X')
camel_df.insert(loc=1, column='hef', value=np.nan)
camel_df.insert(loc=2, column='fou', value=np.nan)
camel_df.insert(loc=3, column='yeo', value=np.nan)

hefzi_df.insert(loc=0, column='cam', value=np.nan)
hefzi_df.insert(loc=1, column='hef', value='X')
hefzi_df.insert(loc=2, column='fou', value=np.nan)
hefzi_df.insert(loc=3, column='yeo', value=np.nan)

fouladiha_df.insert(loc=0, column='cam', value=np.nan)
fouladiha_df.insert(loc=1, column='hef', value=np.nan)
fouladiha_df.insert(loc=2, column='fou', value='X')
fouladiha_df.insert(loc=3, column='yeo', value=np.nan)

yeo_df.insert(loc=0, column='cam', value=np.nan)
yeo_df.insert(loc=1, column='hef', value=np.nan)
yeo_df.insert(loc=2, column='fou', value=np.nan)
yeo_df.insert(loc=3, column='yeo', value='X')


camel_df

,cam,hef,fou,yeo,Reaction Name,Reaction,Reaction Formula,Lower bound,Upper bound,Reaction ID Camels Models,Curation Notes
0,X,NaN,NaN,NaN,glucose transport (uniport),GLCt1r,glc_D_e --> glc_D_c,0.0,NaN,NaN,Unidrectional transporters
1,X,NaN,NaN,NaN,D-glucose transport in via proton symport,GLCt2r,glc_D_e + h_e --> glc_D_c + h_c,0.0,NaN,NaN,Unidrectional transporters
2,X,NaN,NaN,NaN,D-glucose transport in via proton symport,GLCt2_2,glc_D_e + 2.0 h_e --> glc_D_c + 2.0 h_c,0.0,NaN,NaN,Unidrectional transporters
3,X,NaN,NaN,NaN,RE1342,RE1342C,nad_c + sbt_D_c --> glc_D_c + h_c + nadh_c,0.0,NaN,NaN,Unidrectional transporters
4,X,NaN,NaN,NaN,D-fructose transport via sodium cotransport,FRUt4,fru_e + na1_e --> fru_c + na1_c,0.0,NaN,NaN,Unidrectional transporters
...,...,...,...,...,...,...,...,...,...,...,...
600,X,NaN,NaN,NaN,dATP:pyruvate O2-phosphotransferase Purine met...,r0280,dadp_c + h_c + pep_c --> datp_c + pyr_c,NaN,NaN,NaN,EC 3.1.3.49 : present in Rattus norvegicus. Ac...
601,X,NaN,NaN,NaN,"proline oxidase (NAD), mitochondrial",PRO1xm,nad_m + pro_L_m --> 1pyr5c_m + 2.0 h_m + nadh_m,NaN,NaN,Reaction ID kept : PRO1x,Cellular localization in cytosol (https://doi....
602,X,NaN,NaN,NaN,pyrroline-5-carboxylate reductase (m),P5CRxm,1pyr5c_m + 2.0 h_m + nadh_m --> nad_m + pro_L_m,NaN,NaN,Reaction ID kept : P5CRx,Cellular localization in cytosol (https://doi....
603,X,NaN,NaN,NaN,"L-glutamate 5-semialdehyde dehydratase, revers...",G5SADr,glu5sa_c <=> 1pyr5c_c + h2o_c + h_c,NaN,NaN,Reaction ID kept : G5SADrm,L-glutamate 5-semialdehyde dehydratase carry f...


In [4]:
cols = hefzi_df.columns.to_list()+fouladiha_df.columns.to_list()+yeo_df.columns.to_list()+camel_df.columns.to_list()

# Eliminate repetitive values in the 'cols' list
cols = [cols[i] for i in range(len(cols)) if i == cols.index(cols[i])]

cols

['cam',
 'hef',
 'fou',
 'yeo',
 'Reaction',
 'Reaction Name',
 'Subsystem',
 'Reaction Formula',
 'GPR',
 'Curation Notes',
 'References',
 'Genes',
 'Protein',
 'Reversible',
 'Lower bound',
 'Upper bound',
 'Objective',
 'Proteins',
 'EC Number',
 'Mol wt',
 'kcat_forward',
 'kcat_backward',
 'Subsystem (iCHO2291)',
 'Reaction ID Camels Models']

In [5]:
def add_col(df):
    '''
    This function adds the remaining columns from the cols list 
    to an existing df that are not contained in such df
    '''
    df.columns
    add_col = []
    for col in cols:
        if col not in df.columns:
            add_col.append(col)
    df = df.reindex(columns = df.columns.tolist() + add_col)
    return df

In [6]:
# Unify columns for all datasets
hefzi_df = add_col(hefzi_df)
fouladiha_df = add_col(fouladiha_df)
yeo_df = add_col(yeo_df)
camel_df = add_col(camel_df)

camel_df

,cam,hef,fou,yeo,Reaction Name,Reaction,Reaction Formula,Lower bound,Upper bound,Reaction ID Camels Models,...,Genes,Protein,Reversible,Objective,Proteins,EC Number,Mol wt,kcat_forward,kcat_backward,Subsystem (iCHO2291)
0,X,NaN,NaN,NaN,glucose transport (uniport),GLCt1r,glc_D_e --> glc_D_c,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,X,NaN,NaN,NaN,D-glucose transport in via proton symport,GLCt2r,glc_D_e + h_e --> glc_D_c + h_c,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,X,NaN,NaN,NaN,D-glucose transport in via proton symport,GLCt2_2,glc_D_e + 2.0 h_e --> glc_D_c + 2.0 h_c,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,X,NaN,NaN,NaN,RE1342,RE1342C,nad_c + sbt_D_c --> glc_D_c + h_c + nadh_c,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,X,NaN,NaN,NaN,D-fructose transport via sodium cotransport,FRUt4,fru_e + na1_e --> fru_c + na1_c,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,X,NaN,NaN,NaN,dATP:pyruvate O2-phosphotransferase Purine met...,r0280,dadp_c + h_c + pep_c --> datp_c + pyr_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,X,NaN,NaN,NaN,"proline oxidase (NAD), mitochondrial",PRO1xm,nad_m + pro_L_m --> 1pyr5c_m + 2.0 h_m + nadh_m,NaN,NaN,Reaction ID kept : PRO1x,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,X,NaN,NaN,NaN,pyrroline-5-carboxylate reductase (m),P5CRxm,1pyr5c_m + 2.0 h_m + nadh_m --> nad_m + pro_L_m,NaN,NaN,Reaction ID kept : P5CRx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
603,X,NaN,NaN,NaN,"L-glutamate 5-semialdehyde dehydratase, revers...",G5SADr,glu5sa_c <=> 1pyr5c_c + h2o_c + h_c,NaN,NaN,Reaction ID kept : G5SADrm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Reorder columns in all datasets the same way
fouladiha_df = fouladiha_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Subsystem (iCHO2291)', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
yeo_df = yeo_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Subsystem (iCHO2291)', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
hefzi_df = hefzi_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Subsystem (iCHO2291)', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]
camel_df = camel_df[['cam','hef', 'fou', 'yeo', 'Reaction', 'Reaction Name', 'Reaction Formula', 'GPR', 'Subsystem', 'Subsystem (iCHO2291)', 'Genes', 'Protein', 'EC Number', 'Mol wt', 'kcat_forward', 'kcat_backward', 'Reversible','Lower bound', 'Upper bound', 'Objective', 'Curation Notes', 'References', 'Reaction ID Camels Models']]

camel_df

,cam,hef,fou,yeo,Reaction,Reaction Name,Reaction Formula,GPR,Subsystem,Subsystem (iCHO2291),...,Mol wt,kcat_forward,kcat_backward,Reversible,Lower bound,Upper bound,Objective,Curation Notes,References,Reaction ID Camels Models
0,X,NaN,NaN,NaN,GLCt1r,glucose transport (uniport),glc_D_e --> glc_D_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unidrectional transporters,NaN,NaN
1,X,NaN,NaN,NaN,GLCt2r,D-glucose transport in via proton symport,glc_D_e + h_e --> glc_D_c + h_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unidrectional transporters,NaN,NaN
2,X,NaN,NaN,NaN,GLCt2_2,D-glucose transport in via proton symport,glc_D_e + 2.0 h_e --> glc_D_c + 2.0 h_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unidrectional transporters,NaN,NaN
3,X,NaN,NaN,NaN,RE1342C,RE1342,nad_c + sbt_D_c --> glc_D_c + h_c + nadh_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unidrectional transporters,NaN,NaN
4,X,NaN,NaN,NaN,FRUt4,D-fructose transport via sodium cotransport,fru_e + na1_e --> fru_c + na1_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unidrectional transporters,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,X,NaN,NaN,NaN,r0280,dATP:pyruvate O2-phosphotransferase Purine met...,dadp_c + h_c + pep_c --> datp_c + pyr_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EC 3.1.3.49 : present in Rattus norvegicus. Ac...,NaN,NaN
601,X,NaN,NaN,NaN,PRO1xm,"proline oxidase (NAD), mitochondrial",nad_m + pro_L_m --> 1pyr5c_m + 2.0 h_m + nadh_m,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cellular localization in cytosol (https://doi....,NaN,Reaction ID kept : PRO1x
602,X,NaN,NaN,NaN,P5CRxm,pyrroline-5-carboxylate reductase (m),1pyr5c_m + 2.0 h_m + nadh_m --> nad_m + pro_L_m,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cellular localization in cytosol (https://doi....,NaN,Reaction ID kept : P5CRx
603,X,NaN,NaN,NaN,G5SADr,"L-glutamate 5-semialdehyde dehydratase, revers...",glu5sa_c <=> 1pyr5c_c + h2o_c + h_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L-glutamate 5-semialdehyde dehydratase carry f...,NaN,Reaction ID kept : G5SADrm


In [11]:
# Merge all the df into a unified df
all_dfs = pd.concat([camel_df, hefzi_df, fouladiha_df, yeo_df])
all_dfs = all_dfs.reset_index(drop = True)

all_dfs #16901 rows/reactions (many of them repeated)

,cam,hef,fou,yeo,Reaction,Reaction Name,Reaction Formula,GPR,Subsystem,Subsystem (iCHO2291),...,Mol wt,kcat_forward,kcat_backward,Reversible,Lower bound,Upper bound,Objective,Curation Notes,References,Reaction ID Camels Models
0,X,NaN,NaN,NaN,GLCt1r,glucose transport (uniport),glc_D_e --> glc_D_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unidrectional transporters,NaN,NaN
1,X,NaN,NaN,NaN,GLCt2r,D-glucose transport in via proton symport,glc_D_e + h_e --> glc_D_c + h_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unidrectional transporters,NaN,NaN
2,X,NaN,NaN,NaN,GLCt2_2,D-glucose transport in via proton symport,glc_D_e + 2.0 h_e --> glc_D_c + 2.0 h_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unidrectional transporters,NaN,NaN
3,X,NaN,NaN,NaN,RE1342C,RE1342,nad_c + sbt_D_c --> glc_D_c + h_c + nadh_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unidrectional transporters,NaN,NaN
4,X,NaN,NaN,NaN,FRUt4,D-fructose transport via sodium cotransport,fru_e + na1_e --> fru_c + na1_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unidrectional transporters,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17501,NaN,NaN,NaN,X,RTOTALFATPc,NaN,NaN,NaN,R GROUP SYNTHESIS,Exchange/demand/sink reaction,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17502,NaN,NaN,NaN,X,RTOTALt,NaN,NaN,NaN,"TRANSPORT, EXTRACELLULAR",Transport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17503,NaN,NaN,NaN,X,Rtotaltl,NaN,NaN,NaN,"TRANSPORT, LYSOSOMAL",Transport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17504,NaN,NaN,NaN,X,Rtotaltp,NaN,NaN,NaN,"TRANSPORT, PEROXISOMAL",Transport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
def f(x):
    d = {}
    d['GPR'] = x['GPR'].dropna().apply(lambda x: len(x.unique())) > 1
    #d['a_max'] = x['a'].max()
    #d['b_mean'] = x['b'].mean()
    #d['c_d_prodsum'] = (x['c'] * x['d']).sum()
    return pd.Series(d, index=['GPR'])#, 'a_max', 'b_mean', 'c_d_prodsum'])

In [33]:
all_dfs1 = all_dfs.groupby(['Reaction'], group_keys=True).apply(lambda x: f(x))
all_dfs1

AttributeError: 'str' object has no attribute 'unique'

In [10]:
# Group the entire dataset by reaction name 
all_dfs1 = all_dfs.groupby(['Reaction'], group_keys=True).apply(lambda x: x)

# Save merged datasets as Excel file
all_dfs1.to_excel('../Data/all_dfs1.xlsx')

all_dfs1

,,cam,hef,fou,yeo,Reaction,Reaction Name,Reaction Formula,GPR,Subsystem,Subsystem (iCHO2291),...,Mol wt,kcat_forward,kcat_backward,Reversible,Lower bound,Upper bound,Objective,Curation Notes,References,Reaction ID Camels Models
Reaction,,,,,,,,,,,,,,,,,,,,,,
AASAD3m,556,X,NaN,NaN,NaN,AASAD3m,L-aminoadipate-semialdehyde dehydrogenase (NAD...,L2aadp6sa_m + h2o_m + nad_m --> L2aadp_m + 2....,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AASAD3 equivalent human enzyme mainly localize...,NaN,NaN
CAT,571,X,NaN,NaN,NaN,CAT,hydrogen-peroxide:hydrogen-peroxide oxidoreduc...,2.0 h2o2_c --> 2.0 h2o_c + o2_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This enzyme is present in CHO cell but it is s...,NaN,NaN
CATm,569,X,NaN,NaN,NaN,CATm,catalase,2.0 h2o2_m --> 2.0 h2o_m + o2_m,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This enzyme is present in CHO cell but it is s...,NaN,NaN
CATp,570,X,NaN,NaN,NaN,CATp,"catalase A, peroxisomal",2.0 h2o2_x --> 2.0 h2o_x + o2_x,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This enzyme is present in CHO cell but it is s...,NaN,NaN
CYOOm2_cho,575,X,NaN,NaN,NaN,CYOOm2_cho,"cytochrome c oxidase, mitochondrial Complex IV",4.0 focytC_m + 8.0 h_m + o2_m --> 4.0 ficytC_...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Only the reaction generating superoxide anion ...,NaN,Reaction ID kept : CYOOm3_cho
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
r2131,60,X,NaN,NaN,NaN,r2131,Major Facilitator(MFS) TCDB:2.A.1.13.1,bhb_c + but_e --> bhb_e + but_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Compensation for insufficient constraints,NaN,NaN
r2132,61,X,NaN,NaN,NaN,r2132,Major Facilitator(MFS) TCDB:2.A.1.13.1,bhb_c + retn_e --> bhb_e + retn_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Compensation for insufficient constraints,NaN,NaN
r2133,62,X,NaN,NaN,NaN,r2133,Major Facilitator(MFS) TCDB:2.A.1.13.1,2hb_c + bhb_e --> 2hb_e + bhb_c,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Compensation for insufficient constraints,NaN,NaN


In [ ]:
#def f(x):
#    d = {}
#    d['EC Number_m'] = x['EC Number'].first()
#    d['a_max'] = x['a'].max()
#    d['b_mean'] = x['b'].mean()
#    d['c_d_prodsum'] = (x['c'] * x['d']).sum()
#    return pd.Series(d, index=['EC Number_m', 'a_max', 'b_mean', 'c_d_prodsum'])

all_dfs2 = all_dfs.groupby('Reaction').first()
all_dfs2.to_excel('../Data/all_dfs2.xlsx')
all_dfs2

In [ ]:
# this functions are for fetching information from the BiGG database

def get_rxninfo(rxn):

    #download the page
    session = HTMLSession()
    
    response=session.get('http://bigg.ucsd.edu/models/iCHOv1/reactions/'+rxn)
    if response.status_code != 200:
        print('Status code:', response.status_code,f'Failed to fetch info on {rxn} from iCHOv1 model')
        response=session.get('http://bigg.ucsd.edu/universal/reactions/'+rxn)
        if response.status_code != 200:
            print('Status code:', response.status_code,f'Failed to fetch info on {rxn} from BiGG')

    #parse using beautiful soup
    rxn_doc = BeautifulSoup(response.text,'html.parser')

    return rxn_doc


def bigg_attributes(rxn):
    
    '''
    recieves a rxn file from the get_rxninfo function
    and returns Reaction Description, Subsystem
    '''
    if rxn.title.get_text().endswith('iCHOv1'):
        # Reaction description
        rxn_d = rxn.find_all("p")[0].get_text()

        # Reaction formula
        form = rxn.find_all("p")[2].get_text()

        # Subsystem
        subsystem = rxn.find_all("p")[5].get_text()
    else:
        # Reaction description
        rxn_d = rxn.find_all("p")[0].get_text()

        # Reaction formula
        form = rxn.find_all("p")[1].get_text()

        # Subsystem
        subsystem = ''
    
    return rxn_d, form, subsystem


In [ ]:
#all_dfs3 = pd.DataFrame(columns = ['Reactions', 'Reaction Description', 'Reaction Formula', 'Subsystem'])
att = []
for i,r in enumerate(tqdm((all_dfs2.index))):
    try:
        rxn = get_rxninfo(r)
        rxn_d, form, subsystem = bigg_attributes(rxn)
        print(rxn_d, form, subsystem)
        if all_dfs2['Reaction Formula'][i] == None:
            all_dfs2['Reaction Formula'][i] = form
        if all_dfs2['Reaction Name'][i] == None:
            all_dfs2['Reaction Name'][i] = rxn_d
    except:
        print(f'Could not find info for {r}')
        break
        
all_dfs2.to_excel('../Data/all_dfs2.xlsx')

In [ ]:
rxn.find_all("p")

In [ ]:
rxn.find_all("p")[2].get_text()

In [ ]:
rxn.find_all("div", class_="col-lg-8")

In [ ]:
rxn = get_rxninfo('5G2OXPTtx')
print(rxn.title.get_text().endswith('iCHOv1'))


In [ ]:
rxn.find_all("p")[3].get_text()

In [ ]:
for i,r in enumerate(tqdm((all_dfs2.index))):
    if r.endswith('_cho'):
        print(r,i)
        if all_dfs2.index[i-1] == r.split('_')[0]:
            print(all_dfs2.index[i-1],i-1)
        #print(all_dfs2.index[i-1],i-1)